In [1]:
!python3.7 -m pip install h5py

In [2]:
import numpy as np
import h5py

f = h5py.File('nyu_depth_data_labeled.mat')
images = f['images']
labels = f['labels']
depths = f['rawDepths']
print(depths)

<HDF5 dataset "rawDepths": shape (2284, 640, 480), type "<u2">


In [3]:
def normalize_np_array(npa):
    npa = npa.astype('int32')
    normalized = (255*(npa - np.min(npa))/np.ptp(npa)).astype(int)
    return normalized

In [4]:
images_to_test = images[:200]
images_to_test
print(images_to_test[0].shape)
print(np.transpose(images_to_test[0], (1, 2, 0)).shape)

(3, 640, 480)
(640, 480, 3)


In [5]:
#depths_to_test = depths[:200]
#print(depths_to_test)
#normalized_depths_to_test = normalize_np_array(depths_to_test)
#print(normalized_depths_to_test)

In [6]:
from monodepth_model import *
from monodepth_dataloader import *
params = monodepth_parameters(
        encoder='vgg',
        height=256,
        width=512,
        batch_size=2,
        num_threads=1,
        num_epochs=1,
        do_stereo=False,
        wrap_mode="border",
        use_deconv=False,
        alpha_image_loss=0,
        disp_gradient_loss_weight=0,
        lr_loss_weight=0,
        full_summary=False)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [37]:
import scipy
import tensorflow as tf
import tensorflow.contrib.slim as slim
import matplotlib.pyplot as plt

tf.reset_default_graph()

# We have prediction, which is 20 predictions of n x m images.
# targets is the same dimensions.
# To get the root mean squared error, we have to find 
def mean_squared_error(predictions, targets):
    return ((predictions - targets) ** 2).mean()

def post_process_disparity(disp):
    _, h, w = disp.shape
    l_disp = disp[0,:,:]
    r_disp = np.fliplr(disp[1,:,:])
    m_disp = 0.5 * (l_disp + r_disp)
    l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
    l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
    r_mask = np.fliplr(l_mask)
    return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp


def test_simple_with_image_and_labels(params, input_image, labels, checkpoint_path, input_height, input_width, output_name='yikes'):
    """Test function."""

    left  = tf.placeholder(tf.float32, [2, input_height, input_width, 3])
    model = MonodepthModel(params, "test", left, None)
    input_image = np.transpose(input_image, (1, 2, 0))
    original_height, original_width, num_channels = input_image.shape
    input_image = scipy.misc.imresize(input_image, [input_height, input_width], interp='lanczos')
    plt.imsave("raw{}.png".format(output_name), input_image)
    input_image = input_image.astype(np.float32) / 255
    #input_image = np.transpose(input_image, (1, 2, 0))
    print(input_image.shape)
    input_images = np.stack((input_image, np.fliplr(input_image)), 0)

    # SESSION
    config = tf.ConfigProto(allow_soft_placement=True)
    sess = tf.Session(config=config)

    # SAVER
    train_saver = tf.train.Saver()

    # INIT
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coordinator)

    # RESTORE
    restore_path = checkpoint_path.split(".")[0]
    train_saver.restore(sess, restore_path)

    disp = sess.run(model.disp_left_est[0], feed_dict={left: input_images})
    disp_pp = post_process_disparity(disp.squeeze()).astype(np.float32)

    disp_to_img = scipy.misc.imresize(disp_pp.squeeze(), [original_height, original_width])
    resized = np.rot90(scipy.misc.imresize(disp_to_img, [640, 480]), 0)
    plt.imsave("{}.png".format(output_name), resized, cmap='gray')
    # We have it, but resized to original size - let's check what it is
    normalized_prediction = normalize_np_array(disp_to_img)
    return mean_squared_error(normalized_prediction, labels)

In [38]:
mses = []
iterations = 0
max_iterations = 5
for im, lb in zip(images, depths):
    tf.reset_default_graph()
    d = normalize_np_array(lb)
    mses.append(test_simple_with_image_and_labels(params, im, d, '/home/tphan/monodepth/models/model_kitti/model_kitti', 256, 512, iterations))
    print(mses)
    iterations += 1
    if iterations >= max_iterations:
        break
mses

/home/tphan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:32: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


(256, 512, 3)
INFO:tensorflow:Restoring parameters from /home/tphan/monodepth/models/model_kitti/model_kitti


/home/tphan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:59: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.
/home/tphan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


[9426.146214192708]
(256, 512, 3)
INFO:tensorflow:Restoring parameters from /home/tphan/monodepth/models/model_kitti/model_kitti
[9426.146214192708, 15757.810266927083]
(256, 512, 3)
INFO:tensorflow:Restoring parameters from /home/tphan/monodepth/models/model_kitti/model_kitti
[9426.146214192708, 15757.810266927083, 12116.045865885417]
(256, 512, 3)
INFO:tensorflow:Restoring parameters from /home/tphan/monodepth/models/model_kitti/model_kitti
[9426.146214192708, 15757.810266927083, 12116.045865885417, 20233.412747395832]
(256, 512, 3)
INFO:tensorflow:Restoring parameters from /home/tphan/monodepth/models/model_kitti/model_kitti
[9426.146214192708, 15757.810266927083, 12116.045865885417, 20233.412747395832, 16196.133011067708]


[9426.146214192708,
 15757.810266927083,
 12116.045865885417,
 20233.412747395832,
 16196.133011067708]

In [ ]:
mean_of_mses = sum(mses) / len(mses)
import math
print(math.sqrt(mean_of_mses))